In [9]:
# modules
import numpy as np
import matplotlib.pyplot as plt
from   scipy import optimize
import pandas as pd
from scipy.optimize import curve_fit
import scipy.stats as stats
import sys

# Install/import plotly packages- this package has lots of graphical properties
import plotly.graph_objects as go
import plotly.offline as pyo

# Load custom scripts in reusable_code folder
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf

from google.cloud import bigquery
from datetime import date

import re

In [2]:
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/')

In [3]:
# Extracting data to determine LTV
bq = bigquery.Client(project='itv-bde-analytics-prd',credentials=creds)

In [ ]:
#Create your cohorts>> Manual step each time.
#Format 
query="""
with tablea as  (
select distinct * , row_number() over (partition by yearmonth_joined, segment order by yearmonth_joined, yearmonth_sub, segment) as row_num
  from 
  (
  SELECT
    DISTINCT 
    concat(extract(year from account.firststart),extract(month from account.firststart)) as yearmonth_joined,
    concat(extract(year from openEntitlement.start),extract(month from openEntitlement.start)) as yearmonth_sub,
    b.level2 as segment,
    COUNT( DISTINCT britbox_ID ) AS Customers
  FROM
    `itv-bde-analytics-prd.britbox_analytics.entitlements` as a left join `itv-bde-svod-prd.reporting.Sign_up_Funnel_web` as b
    on a.britbox_ID = b.user_ID
        where billingprovider in ('Stripe','iTunes') and date(account.firststart) >= '2019-11-07' and 
        date(openEntitlement.start) <= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 DAY)
  GROUP BY
    1,2,3
    order by 1,2,3)
    where yearmonth_sub is not null
    )
    
  
  (select distinct 
  *, round(customers / max , 2) as pct_remaining from
  
    (select distinct *, max(customers) over (partition by segment, yearmonth_joined ) as max
  from tablea
)
where yearmonth_joined is not null and yearmonth_sub is not null
  order by yearmonth_joined, yearmonth_sub, segment  )
"""
df = bq.query(query)

In [43]:
Get_30Day_Revenues('a','2020-01-01')

In [33]:
str='2020'
re.findall(r"(([12]\d{3})-(0[1-9]|1[012])-(0\d|1\d|2\d|3[01]))", str)


In [ ]:
^(\d|0\d|1\d|2\d|3[01])-(\d|0\d|1[012])-[12]\d{3}$